In [23]:
% load_ext autoreload
% autoreload 2
import numpy as np
import sys
sys.path.append('/Users/IzmailovPavel/Documents/Education/GPproject/gplib/')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Optimization
Testing gplib.optim 
## Deterministic methods

In [78]:
A = np.random.rand(12, 12)
A = A.T.dot(A) + np.eye(12)*0.5
b = np.random.rand(12).reshape((12, 1))
def f(x):
    x = x.reshape((x.size, 1))
    fun = x.T.dot(A.dot(x)) / 2 + b.T.dot(x)
    grad = A.dot(x) + b
    return fun[0, 0], grad.reshape(-1)

In [16]:
x_0 = np.random.rand(12).reshape(12,1)

### Gradient checking

In [35]:
from gplib.optim.utility import check_gradient

In [36]:
check_gradient(f, x_0)


Difference between calculated and approximated gradients
6.70489681598e-06


### FGD with armiho step-size rule

In [29]:
from gplib.optim.methods import fgd
options = {'maxiter': 100, 'verbose':True, 'print_freq':5}
res = fgd(f, x_0, options=options)

Iteration  0 :
	Gradient norm 70.6821328749
	Function value [[ 80.03178287]]
Iteration  5 :
	Gradient norm 61.8215030101
	Function value [[ 60.4918323]]
Iteration  10 :
	Gradient norm 54.0880000021
	Function value [[ 46.044931]]
Iteration  15 :
	Gradient norm 47.3252453752
	Function value [[ 35.09207791]]
Iteration  20 :
	Gradient norm 41.4088444098
	Function value [[ 26.73609989]]
Iteration  25 :
	Gradient norm 36.2323250992
	Function value [[ 20.34954711]]
Iteration  30 :
	Gradient norm 31.7030170031
	Function value [[ 15.46481682]]
Iteration  35 :
	Gradient norm 27.7399520749
	Function value [[ 11.72745281]]
Iteration  40 :
	Gradient norm 24.2723191435
	Function value [[ 8.86736784]]
Iteration  45 :
	Gradient norm 21.238172501
	Function value [[ 6.67834782]]
Iteration  50 :
	Gradient norm 18.583316432
	Function value [[ 5.00279078]]
Iteration  55 :
	Gradient norm 16.260333386
	Function value [[ 3.72017818]]
Iteration  60 :
	Gradient norm 14.2277351689
	Function value [[ 2.7383157]]


### Projected Newton

In [39]:
from gplib.optim.methods import proj_newton

In [52]:
options = {'maxiter': 20, 'print_freq': 5, 'verbose': True}
res = proj_newton(f, x_0.reshape(-1), options=options)

Iteration  0 :
	Gradient norm 70.6821328749
	Function value [[ 80.03178287]]
Iteration  5 :
	Gradient norm 3.81889616913
	Function value [[ 1.9063685]]
Iteration  10 :
	Gradient norm 0.926437709055
	Function value [[-0.40508092]]
Iteration  15 :
	Gradient norm 0.0602986867766
	Function value [[-0.46717017]]


### scipy_wrapper

In [58]:
from gplib.optim.methods import scipy_wrapper

In [59]:
? scipy_wrapper

In [76]:
f(x_0)

(82.62071070935562,
 array([ 16.89237506,  24.64533486,  21.4977129 ,  20.54518979,
         16.23272345,  22.58979195,  15.73096607,  21.31331506,
         24.15904074,  29.12504941,  19.03565451,  17.53973287]))

In [80]:
options = {'maxiter':20}
res = scipy_wrapper(f, x_0.reshape(-1), mydisp=True, print_freq=5, method='L-BFGS-B', jac=True, args=())

Hyper-parameters at iteration 0 : [ 0.90044245  0.98746299  0.95738708  0.04488147  0.22614007]
Hyper-parameters at iteration 5 : [ 0.24481864  0.12418623  0.02725765 -0.44295002 -0.19671216]
Hyper-parameters at iteration 10 : [ 0.19356864 -0.04140271 -0.02882162 -0.46015683  0.16953619]
Hyper-parameters at iteration 15 : [ 0.1984435  -0.04234539 -0.02627296 -0.45865597  0.16965779]
Hyper-parameters at iteration 20 : [ 0.19833996 -0.04245949 -0.02637467 -0.45871676  0.16969607]


## Stochastic methods

In [81]:
? np.random.rand

In [85]:
w = np.random.rand(10, 1)
X = np.random.rand(500, 10)
y = X.dot(w) + np.random.normal(scale=0.1, size=(500,1))

In [145]:
w_0 = np.random.rand(10,1)

In [180]:
def f(point, indices=None):
    point = point.reshape(point.size, 1)
    if indices is None:
        indices = np.arange(y.size).tolist()
    fun = np.linalg.norm(y[indices] - X[indices].dot(point))**2
    grad = -2 * (y[indices] - X[indices].dot(point)).T.dot(X[indices])
    return fun, grad.reshape(-1)

In [150]:
check_gradient(f, w_0)


Difference between calculated and approximated gradients
0.000520222081003


### SGD

In [105]:
from gplib.optim.methods import sgd

In [161]:
def sgd_fun(point, indices=None):
    fun, grad = f(point, indices)
    return grad

In [162]:
options = {'maxiter': 100, 'batch_size':20, 'step0': 1e-3, 'verbose': True}
res = sgd(sgd_fun, np.copy(w_0).reshape(-1), y.size, options=options)

Epoch  0 :
	Step: 0.001
	Parameters [ 0.60204717  0.39577703]
Epoch  10 :
	Step: 0.000281838293126
	Parameters [ 0.57178817  0.27053413]
Epoch  20 :
	Step: 0.000192501227153
	Parameters [ 0.56202173  0.22510289]
Epoch  30 :
	Step: 0.000154022195564
	Parameters [ 0.55647978  0.19802438]
Epoch  40 :
	Step: 0.000131482212883
	Parameters [ 0.55260143  0.17879205]
Epoch  50 :
	Step: 0.000116296460635
	Parameters [ 0.54891719  0.16268472]
Epoch  60 :
	Step: 0.000105200259787
	Parameters [ 0.54756672  0.14979034]
Epoch  70 :
	Step: 9.66487136165e-05
	Parameters [ 0.54538385  0.13951497]
Epoch  80 :
	Step: 8.98049979225e-05
	Parameters [ 0.54359413  0.13094623]
Epoch  90 :
	Step: 8.41718010112e-05
	Parameters [ 0.54295016  0.12270834]


In [167]:
print(f(w)[0])
print(f(w_0)[0])
print(f(res[0])[0])

4.9579283827
161.69038231
5.9864321102


### SAG

In [168]:
from gplib.optim.methods import sag

In [189]:
def sag_fun(point, indices=None):
    fun, grad = f(point, indices)
    return fun / len(indices), grad/len(indices)

In [190]:
options = {'maxiter': 100, 'batch_size':20, 'verbose': True}
res = sag(f, np.copy(w_0).reshape(-1), y.size, options=options)

Epoch  0 :
	Lipschitz constant estimate: 131.598569812
	 [ 0.57756457  0.31940056]
Epoch  10 :
	Lipschitz constant estimate: 131.598569812
	 [ 109.44476933  140.35383993]
Epoch  20 :
	Lipschitz constant estimate: 131.598569812
	 [  992269.2327949  1099695.0089746]
Epoch  30 :
	Lipschitz constant estimate: 131.598569812
	 [  5.57554416e+09   5.89101376e+09]
Epoch  40 :
	Lipschitz constant estimate: 131.598569812
	 [  2.99981776e+13   3.03569497e+13]
Epoch  50 :
	Lipschitz constant estimate: 131.598569812
	 [  1.14358998e+17   1.13554805e+17]
Epoch  60 :
	Lipschitz constant estimate: 131.598569812
	 [  3.87135412e+20   3.74588739e+20]
Epoch  70 :
	Lipschitz constant estimate: 131.598569812
	 [  1.12022132e+24   1.04130848e+24]
Epoch  80 :
	Lipschitz constant estimate: 131.598569812
	 [  2.41601599e+27   2.02963741e+27]
Epoch  90 :
	Lipschitz constant estimate: 131.598569812
	 [  1.02235539e+30  -6.12806931e+29]


In [191]:
f(res[0])

(1.794387989324859e+73,
 array([ -9.26626863e+37,  -9.42963431e+37,  -9.60140599e+37,
         -9.49286996e+37,  -9.78953719e+37,  -9.58515245e+37,
         -9.50518778e+37,  -9.35724414e+37,  -9.67360106e+37,
         -9.47486739e+37]))

### climin_wrapper

In [192]:
from gplib.optim.methods import climin_wrapper

In [193]:
?climin_wrapper

In [198]:
def climin_fun(point, X_tr, y_tr):
    point = point.reshape(point.size, 1)
    fun = np.linalg.norm(y_tr - X_tr.dot(point))**2
    grad = -2 * (y_tr - X_tr.dot(point)).T.dot(X_tr)
    return grad.reshape(-1)

In [200]:
opts = {'maxiter': 100, 'verbose': True, 'batch_size':20, 'step_rate': 0.7, 'decay': 0.8}
res = climin_wrapper(climin_fun, w_0.reshape(-1), X, y, opts, method='AdaDelta')

Using AdaDelta optimizer
Iteration  1 :
	Gradient norm 4.38364241269
Iteration  2 :
	Gradient norm 1.60535813026
Iteration  3 :
	Gradient norm 2.27703856655
Iteration  4 :
	Gradient norm 2.32651802528
Iteration  5 :
	Gradient norm 3.63840664374
Iteration  6 :
	Gradient norm 2.8537127168
Iteration  7 :
	Gradient norm 2.819877695
Iteration  8 :
	Gradient norm 1.03807310698
Iteration  9 :
	Gradient norm 0.845821422092
Iteration  10 :
	Gradient norm 3.36690520721
Iteration  11 :
	Gradient norm 1.96386946527
Iteration  12 :
	Gradient norm 0.6301753459
Iteration  13 :
	Gradient norm 3.10276953906
Iteration  14 :
	Gradient norm 1.91970907458
Iteration  15 :
	Gradient norm 1.84439445425
Iteration  16 :
	Gradient norm 1.60444434426
Iteration  17 :
	Gradient norm 1.137982391
Iteration  18 :
	Gradient norm 1.61941492409
Iteration  19 :
	Gradient norm 2.32449658193
Iteration  20 :
	Gradient norm 1.83719393935
Iteration  21 :
	Gradient norm 3.19372231357
Iteration  22 :
	Gradient norm 2.40740248513

In [202]:
f(res[0])[0]

5.034111864854423

<b style="color:#FF0000";>Problems</b>:
<ul>
    <li> Docstrings for some methods lack or are outdated
    <li> Different methods have differerent names for parameters (especially related to printing progress)
    <li> Bad output format
    <li> Different methods have different requirements for input parameters shape (initial point and gradient)
    <li> SGD method changes the given initial point instead of copying it
    <li> SAG seems to not work
</ul>